In [ ]:
'''
Generate panels for figures 7 and 8 

Maximilian Eggl
Nov 2024
'''


import sys

    
from ExtFuncs import *
%load_ext autoreload
%autoreload 2

import scipy.stats as stats

plt.rc('text', usetex=True)
font = {'family' : 'serif',
        'size'   : 14}
plt.rc('font', **font)
plt.rc('ytick', labelsize=28) 
plt.rc('xtick', labelsize=28)

from sklearn.preprocessing import StandardScaler


plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
from matplotlib.collections import LineCollection
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import davies_bouldin_score
from matplotlib.markers import MarkerStyle
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler

In [ ]:
datDir = './Dat/'
ColorCode = {'D1Nac':np.array([117/255,20/255,124/255]),'D2Nac':np.array([241/255,164/255,84/255]),
             'mPFC':np.array([86/255,187/255,126/255]),'Sus':np.array((0.9682, 0.7208441, 0.612293)),
            'Res':np.array((0.705673158, 0.01555616, 0.150232812)),'nForg':np.array((0.6672529243333334, 0.779176457, 0.992959213)),
            'Forg':np.array((0.2298057, 0.298717966, 0.753683153))}

def PlotSig(left,right,height,starAdjust=0.02,Text=None):
    plt.plot([left,right],[height]*2,c='k')
    if(Text is None):
        plt.plot((left+right)/2,height+starAdjust,marker=(6, 2,0 ),ms=10,c='k')
    else:
        plt.text((left+right)/2,height+starAdjust,Text,ha='center')
        
def PlotBoxScat(A,col,pos,lab=None,ax= None):
    if(ax is None):
        plt.boxplot(A,
                patch_artist=True,
                boxprops=dict(facecolor='none', 
                              color=col),
                capprops=dict(color=col),
                whiskerprops=dict(color=col),
                flierprops=dict(color=col, markeredgecolor=
                                col),
                medianprops=dict(color='g'),
                positions=[pos],
                            showfliers=False
                )  
        plt.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)
    else:
        ax.boxplot(A,
                patch_artist=True,
                boxprops=dict(facecolor='none', 
                              color=col),
                capprops=dict(color=col),
                whiskerprops=dict(color=col),
                flierprops=dict(color=col, markeredgecolor=
                                col),
                medianprops=dict(color='g'),
                positions=[pos],
                            showfliers=False
                )  
        ax.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)
def PlotViolScat(A,col,pos,lab=None,ax= None,hatch=False):
    if(ax is None):
        vp = plt.violinplot(A,
                positions=[pos],widths=0.2,showmeans='True'
                )  
        vp['bodies'][0].set_facecolor(col)
        vp['cbars'].set_color(col)
        vp['cmins'].set_color(col)
        vp['cmaxes'].set_color(col)
        vp['cmeans'].set_color('black')
        plt.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)
        if(hatch):
            vp['bodies'][0].set_hatch('//')
    else:
        vp = ax.violinplot(A,
                positions=[pos],widths=0.2
                )  
        ax.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)
        
def z_score(X):
    # X: ndarray, shape (n_features, n_samples)
    ss = StandardScaler(with_mean=True, with_std=True)
    Xz = ss.fit_transform(X.T).T
    return Xz

In [ ]:
labelsMPC = np.array([1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1])
labelsNAD1 = np.array([1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])
labelsNAD2 = np.array([2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2])

In [ ]:
DatA,TimesA,DatNamesA,TimeCalcDatA,CalcDatA,ShockTimesA,cRawA = GetDat('Batch A 2022 mPFC',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],datDir)
DatB,TimesB,DatNamesB,TimeCalcDatB,CalcDatB,ShockTimesB,cRawB = GetDat('Batch B 2022 mPFC',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],datDir)

DatMPC = DatA+DatB
TimesMPC = np.vstack([TimesA,TimesB])
DatNamesMPC = DatNamesA + DatNamesB
TimeCalcDatMPC = TimeCalcDatA+TimeCalcDatB
CalcDatMPC = CalcDatA+CalcDatB
ShockTimesMPC = np.vstack([ShockTimesA,ShockTimesB])
RawDatMPC = cRawA+cRawB



In [ ]:
FullOrder = []
FullOrderA = []
i=0
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    TC = TimeCalcDatMPC[j][i]
    ST = ShockTimesMPC[j][i]
    Order = np.zeros(Calc.shape[0])
    Order2 = np.zeros(Calc.shape[0])
    Fire0 = Calc[:,TC<ST[0]].mean(axis=1)==0.0
    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order = np.zeros_like(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    FullOrder.append(Order)
    FullOrderA.append(Order2)
    

In [ ]:
j=3
i=0
fig,ax = plt.subplots(figsize=(4,8))
Calc = CalcDatMPC[j][i]
TC = TimeCalcDatMPC[j][i]
ST = ShockTimesMPC[j][i]
FA = FullOrder[j]+FullOrderA[j]
Calc = Calc[~np.isnan(FA)]
FA = FA[~np.isnan(FA)]
Calc = Calc[np.argsort(FA)]
FA = np.sort(FA)#FA
G1 = np.sum(FA<0)
G2 = np.sum(FA<1)
for i,C in enumerate(Calc):
    if(i<G1):
        plt.plot(TC,C/max(C)-i*0.6,c=ColorCode['D1Nac'])
    elif(i>G2):
        plt.plot(TC,C/max(C)-i*0.6,c=ColorCode['D1Nac']+0.3)
    else:
        plt.plot(TC,C/max(C)-i*0.6,c='gray')
        

plt.ylim([-68,2])
plt.yticks([])

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.xticks([0,60,120,180,240,300],fontsize=24)
plt.gca().set_xticks(np.arange(0,340,10), minor=True)
plt.gca().tick_params('x', length=10, width=2, which='major')
plt.gca().tick_params('x', length=5, width=1, which='minor')
plt.gca().set_xlabel('Time [s]',fontsize=32)
plt.ylabel('Normalised Neuronal Traces',fontsize=24)
plt.plot(ST[0],1.4,'v',ms=10,c='k',label='Shock presentation')
plt.plot(ST[1],1.4,'v',ms=10,c='k')

In [ ]:
Corrs = []
fig,ax = plt.subplots(figsize=(12,4))
j = 3
i = 0
R1 = z_score(RawDatMPC[j][i])
R1m = np.nanmean(R1[FullOrder[j]<0],axis=0)
R1a = np.nanmean(R1[FullOrderA[j]>0],axis=0)
R1o = np.nanmean(R1[(FullOrder[j]-FullOrderA[j])==0],axis=0)
TC1 = TimeCalcDatMPC[j][i]
TS1 = TimesMPC[j][i]
T1 = TS1[0]
ST = ShockTimesMPC[j][i]

# Plot the original and aligned time series
plt.plot(TC1,R1m,c=ColorCode['D1Nac'],label='G1 Neurons')
plt.plot(TC1,R1a,c=ColorCode['D1Nac']+0.3,label='G2 Neurons')
plt.plot(TC1,R1o,c='gray',label='Others Neurons')
plt.plot(ST[0],4.2,'v',ms=10,c='k',label='Shock presentation')
plt.plot(ST[1],4.2,'v',ms=10,c='k')
plt.ylabel('z-score',fontsize=24)
plt.xlabel('Time [s]',fontsize=24)
plt.legend(loc=2,frameon=False,fontsize=24,bbox_to_anchor=(0,1.1))
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)   
plt.xticks([0,60,120,180,240,300])
plt.gca().set_xticks(np.arange(0,340,10), minor=True)
plt.gca().tick_params('x', length=10, width=2, which='major')
plt.gca().tick_params('x', length=5, width=1, which='minor')

In [ ]:
j=3
i=0
R = z_score(RawDatMPC[j][i])
TC = TimeCalcDatMPC[j][i]
ST = ShockTimesMPC[j][i]
F = FullOrder[j]
FA = FullOrderA[j]
fig,ax = plt.subplots(1,3,figsize=(12,4))
ax[0].plot(TC,(R)[F<0].T,alpha=0.08,c=ColorCode['D1Nac'])
ax[0].plot(TC,(R)[F<0].T.mean(axis=-1),c=ColorCode['D1Nac'],lw=3)
ax[0].set_xlim([ST[0]-10,ST[0]+10])
ax[0].set_xticks([ST[0]-10,ST[0],ST[0]+10],[-10,'Shock',10])
ax[0].set_ylim(-0.9375260835357497, 6)
ax[1].plot(TC,(R)[FA>0].T,alpha=0.05,c=ColorCode['D1Nac']+0.3)
ax[1].plot(TC,(R)[FA>0].T.mean(axis=-1),c=ColorCode['D1Nac']+0.3,lw=3)
ax[1].set_xlim([ST[0]-10,ST[0]+10])
ax[1].set_xticks([ST[0]-10,ST[0],ST[0]+10],[-10,'Shock',10])
ax[1].get_yaxis().set_visible(False)
ax[1].spines['left'].set_visible(False)
ax[1].set_ylim(-0.9375260835357497, 6)

ax[2].plot(TC,(R)[np.logical_and(F==0,FA==0)].T,alpha=0.05,c='gray')
ax[2].plot(TC,(R)[np.logical_and(F==0,FA==0)].T.mean(axis=-1),c='gray',lw=3)
ax[2].set_xlim([ST[0]-10,ST[0]+10])
ax[2].set_xticks([ST[0]-10,ST[0],ST[0]+10],[-10,'Shock',10])
ax[2].get_yaxis().set_visible(False)
ax[2].spines['left'].set_visible(False)
ax[2].set_ylim(-0.9375260835357497, 6)
fig.suptitle('10 seconds before to 10 seconds after 1st shock \n z-score')
ax[0].set_xlabel('Time [s]',fontsize=24)
ax[1].set_xlabel('Time [s]',fontsize=24)
ax[2].set_xlabel('Time [s]',fontsize=24)

In [ ]:
i=0
CorrShock = []
CorrAShock = []
CorrOShock = []
TCorrs = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    Calc = Calc[~np.isnan(Calc.sum(axis=1))]
    Calc = Calc[Calc.sum(axis=1)>0]
    TC = TimeCalcDatMPC[j][i]
    TS = TimesMPC[j][i]
    ST = ShockTimesMPC[j][i]
    T = TimeCalcDatMPC[j][i]
    D = DatMPC[j][i][2]
    TD = DatMPC[j][i][0]
    FO = FullOrder[j]+FullOrderA[j]
    FO = FO[~np.isnan(FO)]

    mask1 = np.logical_and(T>TS[0],T<TS[1])
    Calc = Calc[:,mask1]
    T = T[mask1]-TS[0]
    mask2 = np.logical_and(T>TD[0],T<TD[-1])

    Calc = Calc[:,mask2]
    Calc = Calc[Calc.sum(axis=1)>0]
    T = T[mask2]

    f = sp.interpolate.interp1d(TD,D)
    D_int = f(T)

    Corrs = []
    for C in Calc:
        Corrs.append(np.corrcoef(D_int,C)[0,1])
    TCorrs.append(np.array(Corrs))
    CorrShock.append(np.array(Corrs)[FO<0])
    CorrOShock.append(np.array(Corrs)[FO==0])
    CorrAShock.append(np.array(Corrs)[FO>0])
    
AntiO = []
SpikeO = []
AntiSpike = []
DB1 = []
DB2 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][0][~np.isnan(CalcDatMPC[j][0].sum(axis=1))]
    O    = (FullOrder[j]+FullOrderA[j])[~np.isnan(CalcDatMPC[j][0].sum(axis=1))].clip(-1,1)
    Cr = np.zeros_like(TCorrs[j])
    Cr[TCorrs[j]<-0.1] = -1
    Cr[TCorrs[j]>0.1]  = 1
    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T
    db_index = davies_bouldin_score(Xp.T, O)
    DB1.append(db_index)

    colors = [ColorCode['D1Nac'],'gray',ColorCode['D1Nac']+0.3]
    cmap = ListedColormap(colors)
    colors2 = ['blue','black','red']
    cmap2 = ListedColormap(colors2)
    if(np.isnan(TCorrs[j]).all()):
        DB2.append(math.nan)
    else:
        db_index = davies_bouldin_score(Xp.T, Cr)
        DB2.append(db_index)

    if(j==7):
        variance_explained = pca.explained_variance_ratio_
        print("Variance explained by each component:", variance_explained*100)
        scatter = plt.scatter(Xp[0][Cr==-1],-Xp[1][Cr==-1],color='blue',marker=MarkerStyle("o", fillstyle="right"),
           s=100,label='-ve Corr. w/ freezing')
        plt.scatter(Xp[0][Cr==0],-Xp[1][Cr==0],color='black',marker=MarkerStyle("o", fillstyle="right"),
           s=100,label='Not Corr. w/ freezing')
        plt.scatter(Xp[0][Cr==1],-Xp[1][Cr==1],color='red',marker=MarkerStyle("o", fillstyle="right"),
           s=100,label='+ve Corr. w/ freezing')
        plt.scatter(Xp[0][O==-1],-Xp[1][O==-1],color=ColorCode['D1Nac'],cmap=cmap,marker=MarkerStyle("o", fillstyle="left"),
                   s=100,label='G1 Neurons')
        plt.scatter(Xp[0][O==0],-Xp[1][O==0],color='gray',cmap=cmap,marker=MarkerStyle("o", fillstyle="left"),
                   s=100,label='Other')
        plt.scatter(Xp[0][O==1],-Xp[1][O==1],color=ColorCode['D1Nac']+0.3,cmap=cmap,marker=MarkerStyle("o", fillstyle="left"),
                   s=100,label='G2 Neurons')
        plt.xlabel('PC 1 \n [9.9\% variance explained]',fontsize=32)
        plt.ylabel('PC 2 \n [6.3\% variance explained]',fontsize=32)
        plt.legend(ncol=2,loc=1,bbox_to_anchor=(1,1.6),fontsize=28,
                   columnspacing=0.3,handlelength=0.4,handletextpad=0.1)
    plt.show()

In [ ]:
K = np.array([np.nanmean(C) for C in CorrShock])
K = K[~np.isnan(K)]
K2 = np.array([np.nanmean(C) for C in CorrAShock])
K2 = K2[~np.isnan(K2)]
K3 = np.array([np.nanmean(C) for C in CorrOShock])
K3 = K3[~np.isnan(K3)]
PlotViolScat(K,np.clip(ColorCode['D1Nac'],0,1),1)
PlotViolScat(K2,np.clip(ColorCode['D1Nac']+0.3,0,1),1.5)
PlotViolScat(K3,'gray',2)
plt.xticks([1,1.5,2],['G1','G2','Other'])
plt.axhline(0,c='k',ls='--')
plt.plot([1.5,1.5],[0.18,0.2],'k')
plt.plot([2,2],[0.18,0.2],'k')

x = [1.98, 2.02]
y = [0.22,  0.22]
plt.plot(x, y, marker=(6, 2, 0), ms=10, c='k', linestyle='') 

x = np.array([0.94,0.98, 1.02,1.06])+0.5
y = [0.22]*4
plt.plot(x, y, marker=(6, 2, 0), ms=10, c='k', linestyle='') 


In [ ]:
print('-'*10 + 'Done 1'+'-'*10)    
i = 1
FullOrder2 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrder[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderA[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder2.append(Order)
print('-'*10 + 'Done 2'+'-'*10)
i = 2
FullOrder3 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder3.append(Order)
print('-'*10 + 'Done 3'+'-'*10)
i = 3
FullOrder4 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder3[j]<0,FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder3[j]>0,FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder4.append(Order)
print('-'*10 + 'Done 4'+'-'*10)

In [ ]:
A = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder,dtype=object)[labelsMPC==2]]
A2 = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder,dtype=object)[labelsMPC==1]]

PlotViolScat(A,np.clip(ColorCode['D1Nac'],0,1),1,lab='R+')
PlotViolScat(A2,np.clip(ColorCode['D1Nac']+0.3,0,1),1.2,lab='R-')

A_b = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder2,dtype=object)[labelsMPC==2]]
A2_b = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder2,dtype=object)[labelsMPC==1]]

PlotViolScat(A_b,np.clip(ColorCode['D1Nac'],0,1),2)
PlotViolScat(A2_b,np.clip(ColorCode['D1Nac']+0.3,0,1),2.2)


A_c = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder3,dtype=object)[labelsMPC==2]]
A2_c = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder3,dtype=object)[labelsMPC==1]]

PlotViolScat(A_c,np.clip(ColorCode['D1Nac'],0,1),3)
PlotViolScat(A2_c,np.clip(ColorCode['D1Nac']+0.3,0,1),3.2)


A_d = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder4,dtype=object)[labelsMPC==2]]
A2_d = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder4,dtype=object)[labelsMPC==1]]

PlotViolScat(A_d,np.clip(ColorCode['D1Nac'],0,1),4)
PlotViolScat(A2_d,np.clip(ColorCode['D1Nac']+0.3,0,1),4.2)


PlotSig(1,1.2,0.35)
PlotSig(2,2.2,0.42)
PlotSig(3,3.2,0.42)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
plt.ylabel('Proportion of G1 in PL [\%]',fontsize=32)

plt.legend(fontsize=28,handletextpad=-0.5,markerscale=2,ncols = 2,loc=1,
           bbox_to_anchor=(0.8,1.2))

In [ ]:
i=0
zPre = []
z1   = []
z2   = []
for j in range(len(labelsMPC)):
    Calc = CalcDatMPC[j][i]
    Order = FullOrder[j][~np.isnan(Calc.sum(axis=1))]
    Calc = Calc[~np.isnan(Calc.sum(axis=1))]
    Calc = Calc[Calc.sum(axis=1)>0]
    TC = TimeCalcDatMPC[j][i]
    TS = TimesMPC[j][i]
    ST = ShockTimesMPC[j][i]
    D = DatMPC[j][i]
    C1 = z_score(Calc[Order<0])
    C1_pre = C1[:,(TC>ST[0]-2)*(TC<ST[0])]
    C1_1 = C1[:,(TC>ST[0])*(TC<ST[0]+2)]
    C1_2 = C1[:,(TC>ST[1])*(TC<ST[1]+2)]
    zPre.append(np.mean(C1_pre))
    z1.append(np.mean(C1_1))
    z2.append(np.mean(C1_2))
zPre = np.array(zPre)
z1 = np.array(z1)
z2 = np.array(z2)

In [ ]:
A = z1[labelsMPC==2]
A2 = z1[labelsMPC==1]

PlotViolScat(A,np.clip(ColorCode['D1Nac'],0,1),1,lab='R+')
PlotViolScat(A2,np.clip(ColorCode['D1Nac']+0.3,0,1),2,lab='R-')

A = z2[labelsMPC==2]
A2 = z2[labelsMPC==1]

PlotViolScat(A,np.clip(ColorCode['D1Nac'],0,1),1.2)
PlotViolScat(A2,np.clip(ColorCode['D1Nac']+0.3,0,1),2.2)
PlotSig(1,1.2,4.2,0.1)
plt.ylabel('Avg. z-score 2s after shock',fontsize=28)
plt.xticks([1,1.2,2,2.2],['1st','2nd']*2,fontsize=20)

plt.legend(fontsize=28,handletextpad=-0.5,markerscale=2,ncols = 2,loc=1,
           bbox_to_anchor=(0.8,1.2))

In [ ]:
AntiO = []
SpikeO = []
AntiSpike = []
DB = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][0][~np.isnan(CalcDatMPC[j][0].sum(axis=1))]
    O    = (FullOrder[j]+FullOrderA[j])[~np.isnan(CalcDatMPC[j][0].sum(axis=1))].clip(-1,1)

    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T
    db_index = davies_bouldin_score(Xp.T, O)
    DB.append(db_index)

    #plt.title(LargNames[labelsMPC[j]-1])
    X1 = Xp[:,O==1].mean(axis=-1)
    X2 = Xp[:,O==0].mean(axis=-1)
    X3 = Xp[:,O==-1].mean(axis=-1)
    maxdist = 0
    for x in Xp.T:
        for y in Xp.T:
            maxdist = max(np.linalg.norm(x-y),maxdist)
    AntiO.append(np.linalg.norm(X1-X2)/maxdist)
    AntiSpike.append(np.linalg.norm(X1-X3)/maxdist)
    SpikeO.append(np.linalg.norm(X2-X3)/maxdist)

    
AntiO = []
SpikeO = []
AntiSpike = []
DB1 = []
kk = 1
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][kk][~np.isnan(CalcDatMPC[j][kk].sum(axis=1))]
    O    = FullOrder2[j][~np.isnan(CalcDatMPC[j][kk].sum(axis=1))].clip(-1,1)

    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T

    colors = [ColorCode['D2Nac'],'gray',np.clip(ColorCode['D2Nac']+0.2,0,1)]
    cmap = ListedColormap(colors)
    
    db_index = davies_bouldin_score(Xp.T, O)
    DB1.append(db_index)
    X1 = Xp[:,O==1].mean(axis=-1)
    X2 = Xp[:,O==0].mean(axis=-1)
    X3 = Xp[:,O==-1].mean(axis=-1)
    maxdist = 0
    for x in Xp.T:
        for y in Xp.T:
            maxdist = max(np.linalg.norm(x-y),maxdist)
    AntiO.append(np.linalg.norm(X1-X2)/maxdist)
    AntiSpike.append(np.linalg.norm(X1-X3)/maxdist)
    SpikeO.append(np.linalg.norm(X2-X3)/maxdist)


AntiO = []
SpikeO = []
AntiSpike = []
DB2 = []
kk = 2
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][kk][~np.isnan(CalcDatMPC[j][kk].sum(axis=1))]
    O    = FullOrder3[j][~np.isnan(CalcDatMPC[j][kk].sum(axis=1))].clip(-1,1)

    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T

    colors = [ColorCode['D2Nac'],'gray',np.clip(ColorCode['D2Nac']+0.2,0,1)]
    cmap = ListedColormap(colors)
    
    db_index = davies_bouldin_score(Xp.T, O)
    DB2.append(db_index)
    X1 = Xp[:,O==1].mean(axis=-1)
    X2 = Xp[:,O==0].mean(axis=-1)
    X3 = Xp[:,O==-1].mean(axis=-1)
    maxdist = 0
    for x in Xp.T:
        for y in Xp.T:
            maxdist = max(np.linalg.norm(x-y),maxdist)
    AntiO.append(np.linalg.norm(X1-X2)/maxdist)
    AntiSpike.append(np.linalg.norm(X1-X3)/maxdist)
    SpikeO.append(np.linalg.norm(X2-X3)/maxdist)


AntiO = []
SpikeO = []
AntiSpike = []
DB3 = []
kk = 3
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][kk][~np.isnan(CalcDatMPC[j][kk].sum(axis=1))]
    O    = FullOrder4[j][~np.isnan(CalcDatMPC[j][kk].sum(axis=1))].clip(-1,1)

    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T

    colors = [ColorCode['D2Nac'],'gray',np.clip(ColorCode['D2Nac']+0.2,0,1)]
    cmap = ListedColormap(colors)
    
    db_index = davies_bouldin_score(Xp.T, O)
    DB3.append(db_index)
    X1 = Xp[:,O==1].mean(axis=-1)
    X2 = Xp[:,O==0].mean(axis=-1)
    X3 = Xp[:,O==-1].mean(axis=-1)
    maxdist = 0
    for x in Xp.T:
        for y in Xp.T:
            maxdist = max(np.linalg.norm(x-y),maxdist)
    AntiO.append(np.linalg.norm(X1-X2)/maxdist)
    AntiSpike.append(np.linalg.norm(X1-X3)/maxdist)
    SpikeO.append(np.linalg.norm(X2-X3)/maxdist)
    
DR =np.vstack([np.array(DB)[labelsMPC==2],np.array(DB1)[labelsMPC==2],np.array(DB2)[labelsMPC==2],np.array(DB3)[labelsMPC==2]])
DS =np.vstack([np.array(DB)[labelsMPC==1],np.array(DB1)[labelsMPC==1],np.array(DB2)[labelsMPC==1],np.array(DB3)[labelsMPC==1]])

plt.errorbar([1,2,3,4],DR.mean(axis=1),stats.sem(DR,axis=1),
             c=ColorCode['D1Nac']+0.3,label='R+',lw=3)
plt.errorbar([1,2,3,4],DS.mean(axis=1),stats.sem(DS,axis=1),
             c=ColorCode['D1Nac'],label='R-',lw=3)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
plt.ylabel('Davies-Bouldin Index \n for PC space',fontsize=28)
plt.legend(fontsize=28,loc=4,bbox_to_anchor=(0.9,0))

plt.plot((4,4),(4.8,5.05),'k')
plt.plot([3.95,4.05],[5.2,5.2],marker=(6, 2,0 ),ms=10,c='k',linestyle='') 

In [ ]:
DatC,TimesC,DatNamesC,TimeCalcDatC,CalcDatC,ShockTimesC,rawC = GetDat('Batch C 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=1)
DatD,TimesD,DatNamesD,TimeCalcDatD,CalcDatD,ShockTimesD,rawD = GetDat('Batch D 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=1)
DatNAD1 = DatC+DatD
TimesNAD1 = np.vstack([TimesC,TimesD])
DatNamesNAD1 = DatNamesC + DatNamesD
TimeCalcDatNAD1 = TimeCalcDatC+TimeCalcDatD
CalcDatNAD1 = CalcDatC+CalcDatD
ShockTimesNAD1 = np.vstack([ShockTimesC,ShockTimesD])
RawDatNAD1 = rawC+rawD

In [ ]:
FullOrder = []
FullOrderA = []
i=0
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    TC = TimeCalcDatNAD1[j][i]
    ST = ShockTimesNAD1[j][i]

    SV = np.zeros_like(TC)
    A = np.argmin(np.abs(TC-ST[0]))
    B = np.argmin(np.abs(TC-ST[1]))

    SV[A:A+100] = 1
    SV[B:B+100] = 1

    MaxCorrV = []
    for C in Calc:
        MaxCorrV.append(np.corrcoef(SV,C)[0,1])
    MaxCorrV = np.array(MaxCorrV)

    Order = np.zeros(Calc.shape[0]).astype(np.float64)
    Order2 = np.zeros(Calc.shape[0]).astype(np.float64)
    Fire0 = MaxCorrV>0.2


    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    Order2[Order<0] = 0

    FullOrder.append(Order)
    FullOrderA.append(Order2)

print('1 done')

i = 1
FullOrder2 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrder[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderA[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder2.append(Order)

print('2 done')

i = 2
FullOrder3 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder3.append(Order)
    
print('3 done')

i = 3
FullOrder4 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder3[j]<0,FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder3[j]>0,FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder4.append(Order)
    
print('4 done')

In [ ]:
A = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder,dtype=object)[labelsNAD1==2]]
A2 = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder,dtype=object)[labelsNAD1==1]]
p1 = stats.ttest_ind(A,A2).pvalue
PlotViolScat(A,np.clip(ColorCode['D2Nac'],0,1),1,lab='R+')
PlotViolScat(A2,np.clip(ColorCode['D2Nac']+0.2,0,1),1.2,lab='R-')

A = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder2,dtype=object)[labelsNAD1==2]]
A2 = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder2,dtype=object)[labelsNAD1==1]]
p2 = stats.ttest_ind(A,A2).pvalue
PlotViolScat(A,np.clip(ColorCode['D2Nac'],0,1),2)
PlotViolScat(A2,np.clip(ColorCode['D2Nac']+0.2,0,1),2.2)


A = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder3,dtype=object)[labelsNAD1==2]]
A2 = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder3,dtype=object)[labelsNAD1==1]]
p3 = stats.ttest_ind(A,A2).pvalue
PlotViolScat(A,np.clip(ColorCode['D2Nac'],0,1),3)
PlotViolScat(A2,np.clip(ColorCode['D2Nac']+0.2,0,1),3.2)


A = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder4,dtype=object)[labelsNAD1==2]]
A2 = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder4,dtype=object)[labelsNAD1==1]]
p4 = stats.ttest_ind(A,A2).pvalue
PlotViolScat(A,np.clip(ColorCode['D2Nac'],0,1),4)
PlotViolScat(A2,np.clip(ColorCode['D2Nac']+0.2,0,1),4.2)


#PlotSig(1,1.2,0.35)
#PlotSig(2,2.2,0.42)
#PlotSig(3,3.2,0.42)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
plt.ylabel('Proportion of G1 in PL [\%]',fontsize=32)
plt.legend(fontsize=28,handletextpad=-0.5,markerscale=2,ncols = 2,loc=1,
           bbox_to_anchor=(0.8,1.2))
PlotSig(3,3.2,1)
PlotSig(4,4.2,1)

In [ ]:
j=0
i=0
R = RawDatNAD1[j][i]
TC = TimeCalcDatNAD1[j][i]
ST = ShockTimesNAD1[j][i]
F = FullOrder[i]
FA = FullOrderA[i]
fig,ax = plt.subplots(1,3,figsize=(12,4))
ax[0].plot(TC,(R)[F<0].T,alpha=0.08,c=ColorCode['D2Nac'])
ax[0].plot(TC,(R)[F<0].T.mean(axis=-1),c=ColorCode['D2Nac'],lw=3)
ax[0].set_xlim([ST[0]-10,ST[0]+10])
ax[0].set_xticks([ST[0]-10,ST[0],ST[0]+10],[-10,'Shock',10])
ax[0].set_ylim(-3, 20)
ax[1].plot(TC,(R)[FA>0].T,alpha=0.05,c=np.clip(ColorCode['D2Nac']+0.2,0,1))
ax[1].plot(TC,(R)[FA>0].T.mean(axis=-1),c=np.clip(ColorCode['D2Nac'],0,1),lw=3)
ax[1].set_xlim([ST[0]-10,ST[0]+10])
ax[1].set_xticks([ST[0]-10,ST[0],ST[0]+10],[-10,'Shock',10])
ax[1].get_yaxis().set_visible(False)
ax[1].spines['left'].set_visible(False)
ax[1].set_ylim(-3, 20)

ax[2].plot(TC,(R)[np.logical_and(F==0,FA==0)].T,alpha=0.05,c='gray')
ax[2].plot(TC,(R)[np.logical_and(F==0,FA==0)].T.mean(axis=-1),c='gray',lw=3)
ax[2].set_xlim([ST[0]-10,ST[0]+10])
ax[2].set_xticks([ST[0]-10,ST[0],ST[0]+10],[-10,'Shock',10])
ax[2].get_yaxis().set_visible(False)
ax[2].spines['left'].set_visible(False)
ax[2].set_ylim(-3, 20)
fig.suptitle('10 seconds before to 10 seconds after 1st shock \n z-score')
ax[0].set_xlabel('Time [s]',fontsize=24)
ax[1].set_xlabel('Time [s]',fontsize=24)
ax[2].set_xlabel('Time [s]',fontsize=24)

In [ ]:
K = np.array([np.nanmean(C) for C in CorrShock])
K = K[~np.isnan(K)]
K2 = np.array([np.nanmean(C) for C in CorrAShock])
K2 = K2[~np.isnan(K2)]
K3 = np.array([np.nanmean(C) for C in CorrOShock])
K3 = K3[~np.isnan(K3)]
PlotViolScat(K,np.clip(ColorCode['D2Nac'],0,1),1)
PlotViolScat(K2,np.clip(ColorCode['D2Nac']+0.2,0,1),1.5)
PlotViolScat(K3,'gray',2)
plt.xticks([1,1.5,2],['G1','G2','Other'])
plt.axhline(0,c='k',ls='--')
plt.plot([1.5,1.5],[0.18,0.2],'k')

x = np.array([0.94,0.98, 1.02,1.06])+0.5
y = [0.22]*4
plt.plot(x, y, marker=(6, 2, 0), ms=10, c='k', linestyle='') 


plt.ylabel('Correlation with Freezing',fontsize=28)

In [ ]:
i=0
zPre = []
z1   = []
z2   = []
for j in range(len(labelsNAD1)):
    Calc = CalcDatNAD1[j][i]
    Order = FullOrder[j][~np.isnan(Calc.sum(axis=1))]
    Calc = Calc[~np.isnan(Calc.sum(axis=1))]
    Calc = Calc[Calc.sum(axis=1)>0]
    TC = TimeCalcDatNAD1[j][i]
    TS = TimesNAD1[j][i]
    ST = ShockTimesNAD1[j][i]
    D = DatNAD1[j][i]
    C1 = z_score(Calc[Order<0])
    C1_pre = C1[:,(TC>ST[0]-2)*(TC<ST[0])]
    C1_1 = C1[:,(TC>ST[0])*(TC<ST[0]+2)]
    C1_2 = C1[:,(TC>ST[1])*(TC<ST[1]+2)]
    zPre.append(np.mean(C1_pre))
    z1.append(np.mean(C1_1))
    z2.append(np.mean(C1_2))
zPre = np.array(zPre)
z1 = np.array(z1)
z2 = np.array(z2)
A = z1[labelsNAD1==2]
A2 = z1[labelsNAD1==1]

PlotViolScat(A,np.clip(ColorCode['D2Nac'],0,1),1,lab='R+')
PlotViolScat(A2,np.clip(ColorCode['D2Nac']+0.2,0,1),2,lab='R-')

A = z2[labelsNAD1==2]
A2 = z2[labelsNAD1==1]

PlotViolScat(A,np.clip(ColorCode['D2Nac'],0,1),1.2)
PlotViolScat(A2,np.clip(ColorCode['D2Nac']+0.2,0,1),2.2)

plt.ylabel('Avg. z-score 2s after shock',fontsize=28)
plt.xticks([1,1.2,2,2.2],['1st','2nd']*2,fontsize=20)

plt.legend(fontsize=28,handletextpad=-0.5,markerscale=2,ncols = 2,loc=1,
           bbox_to_anchor=(0.8,1.2))


In [ ]:
AntiO = []
SpikeO = []
AntiSpike = []
DB = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][0][~np.isnan(CalcDatNAD1[j][0].sum(axis=1))]
    O    = (FullOrder[j]+FullOrderA[j])[~np.isnan(CalcDatNAD1[j][0].sum(axis=1))].clip(-1,1)

    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T
    db_index = davies_bouldin_score(Xp.T, O)
    DB.append(db_index)

    #plt.title(LargNames[labelsNAD1[j]-1])
    X1 = Xp[:,O==1].mean(axis=-1)
    X2 = Xp[:,O==0].mean(axis=-1)
    X3 = Xp[:,O==-1].mean(axis=-1)
    maxdist = 0
    for x in Xp.T:
        for y in Xp.T:
            maxdist = max(np.linalg.norm(x-y),maxdist)
    AntiO.append(np.linalg.norm(X1-X2)/maxdist)
    AntiSpike.append(np.linalg.norm(X1-X3)/maxdist)
    SpikeO.append(np.linalg.norm(X2-X3)/maxdist)

    
AntiO = []
SpikeO = []
AntiSpike = []
DB1 = []
kk = 1
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][kk][~np.isnan(CalcDatNAD1[j][kk].sum(axis=1))]
    O    = FullOrder2[j][~np.isnan(CalcDatNAD1[j][kk].sum(axis=1))].clip(-1,1)

    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T

    colors = [ColorCode['D2Nac'],'gray',np.clip(ColorCode['D2Nac']+0.2,0,1)]
    cmap = ListedColormap(colors)
    
    db_index = davies_bouldin_score(Xp.T, O)
    DB1.append(db_index)
    X1 = Xp[:,O==1].mean(axis=-1)
    X2 = Xp[:,O==0].mean(axis=-1)
    X3 = Xp[:,O==-1].mean(axis=-1)
    maxdist = 0
    for x in Xp.T:
        for y in Xp.T:
            maxdist = max(np.linalg.norm(x-y),maxdist)
    AntiO.append(np.linalg.norm(X1-X2)/maxdist)
    AntiSpike.append(np.linalg.norm(X1-X3)/maxdist)
    SpikeO.append(np.linalg.norm(X2-X3)/maxdist)


AntiO = []
SpikeO = []
AntiSpike = []
DB2 = []
kk = 2
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][kk][~np.isnan(CalcDatNAD1[j][kk].sum(axis=1))]
    O    = FullOrder3[j][~np.isnan(CalcDatNAD1[j][kk].sum(axis=1))].clip(-1,1)

    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T

    colors = [ColorCode['D2Nac'],'gray',np.clip(ColorCode['D2Nac']+0.2,0,1)]
    cmap = ListedColormap(colors)
    
    db_index = davies_bouldin_score(Xp.T, O)
    DB2.append(db_index)
    X1 = Xp[:,O==1].mean(axis=-1)
    X2 = Xp[:,O==0].mean(axis=-1)
    X3 = Xp[:,O==-1].mean(axis=-1)
    maxdist = 0
    for x in Xp.T:
        for y in Xp.T:
            maxdist = max(np.linalg.norm(x-y),maxdist)
    AntiO.append(np.linalg.norm(X1-X2)/maxdist)
    AntiSpike.append(np.linalg.norm(X1-X3)/maxdist)
    SpikeO.append(np.linalg.norm(X2-X3)/maxdist)


AntiO = []
SpikeO = []
AntiSpike = []
DB3 = []
kk = 3
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][kk][~np.isnan(CalcDatNAD1[j][kk].sum(axis=1))]
    O    = FullOrder4[j][~np.isnan(CalcDatNAD1[j][kk].sum(axis=1))].clip(-1,1)

    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T

    colors = [ColorCode['D2Nac'],'gray',np.clip(ColorCode['D2Nac']+0.2,0,1)]
    cmap = ListedColormap(colors)
    
    db_index = davies_bouldin_score(Xp.T, O)
    DB3.append(db_index)
    X2 = Xp[:,O==0].mean(axis=-1)
    X3 = Xp[:,O==-1].mean(axis=-1)
    maxdist = 0
    for x in Xp.T:
        for y in Xp.T:
            maxdist = max(np.linalg.norm(x-y),maxdist)
    AntiO.append(np.linalg.norm(X1-X2)/maxdist)
    AntiSpike.append(np.linalg.norm(X1-X3)/maxdist)
    SpikeO.append(np.linalg.norm(X2-X3)/maxdist)
    
DR =np.vstack([np.array(DB)[labelsNAD1==2],np.array(DB1)[labelsNAD1==2],np.array(DB2)[labelsNAD1==2],np.array(DB3)[labelsNAD1==2]])
DS =np.vstack([np.array(DB)[labelsNAD1==1],np.array(DB1)[labelsNAD1==1],np.array(DB2)[labelsNAD1==1],np.array(DB3)[labelsNAD1==1]])

plt.errorbar([1,2,3,4],DR.mean(axis=1),stats.sem(DR,axis=1),
             c=np.clip(ColorCode['D2Nac']+0.2,0,1),label='R+',lw=3)
plt.errorbar([1,2,3,4],DS.mean(axis=1),stats.sem(DS,axis=1),
             c=ColorCode['D2Nac'],label='R-',lw=3)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
plt.ylabel('Davies-Bouldin Index \n for PC space',fontsize=28)
plt.legend(fontsize=28,loc=4,bbox_to_anchor=(0.9,0))
plt.plot([4,4],[6,5.8],lw=2,c='k')
plt.text(4,6.1,'$p=0.07$',ha='center',fontsize=24)
plt.plot([3,3],[6,5.8],lw=2,c='k')
plt.text(3,6.1,'$p=0.08$',ha='center',fontsize=24)

In [ ]:
DatC,TimesC,DatNamesC,TimeCalcDatC,CalcDatC,ShockTimesC,rawC = GetDat('Batch E 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=1)
DatD,TimesD,DatNamesD,TimeCalcDatD,CalcDatD,ShockTimesD,rawD = GetDat('Batch F 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=1)
DatNAD2 = DatC+DatD
TimesNAD2 = np.vstack([TimesC,TimesD])
DatNamesNAD2 = DatNamesC + DatNamesD
TimeCalcDatNAD2 = TimeCalcDatC+TimeCalcDatD
CalcDatNAD2 = CalcDatC+CalcDatD
ShockTimesNAD2 = np.vstack([ShockTimesC,ShockTimesD])
RawDatNAD2 = rawC+rawD

In [ ]:
FullOrder = []
FullOrderA = []
i=0
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    TC = TimeCalcDatNAD2[j][i]
    ST = ShockTimesNAD2[j][i]

    SV = np.zeros_like(TC)
    A = np.argmin(np.abs(TC-ST[0]))
    B = np.argmin(np.abs(TC-ST[1]))

    SV[A:A+100] = 1
    SV[B:B+100] = 1

    MaxCorrV = []
    for C in Calc:
        MaxCorrV.append(np.corrcoef(SV,C)[0,1])
    MaxCorrV = np.array(MaxCorrV)

    Order = np.zeros(Calc.shape[0]).astype(np.float64)
    Order2 = np.zeros(Calc.shape[0]).astype(np.float64)
    Fire0 = MaxCorrV>0.2


    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    Order2[Order<0] = 0

    FullOrder.append(Order)
    FullOrderA.append(Order2)
i = 1
FullOrder2 = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrder[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderA[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder2.append(Order)

i = 2
FullOrder3 = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder3.append(Order)

i = 3
FullOrder4 = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder3[j]<0,FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder3[j]>0,FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder4.append(Order)

In [ ]:
A = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder,dtype=object)[labelsNAD2==2]]
A2 = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder,dtype=object)[labelsNAD2==1]]
print(stats.ttest_ind(A,A2))
PlotViolScat(A,np.clip(ColorCode['mPFC'],0,1),1,lab='R+')
PlotViolScat(A2,np.clip(ColorCode['mPFC']+0.2,0,1),1.2,lab='R-')

A = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder2,dtype=object)[labelsNAD2==2]]
A2 = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder2,dtype=object)[labelsNAD2==1]]
print(stats.ttest_ind(A,A2))
PlotViolScat(A,np.clip(ColorCode['mPFC'],0,1),2)
PlotViolScat(A2,np.clip(ColorCode['mPFC']+0.2,0,1),2.2)


A = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder3,dtype=object)[labelsNAD2==2]]
A2 = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder3,dtype=object)[labelsNAD2==1]]
print(stats.ttest_ind(A,A2))
PlotViolScat(A,np.clip(ColorCode['mPFC'],0,1),3)
PlotViolScat(A2,np.clip(ColorCode['mPFC']+0.2,0,1),3.2)


A = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder4,dtype=object)[labelsNAD2==2]]
A2 = [np.sum(F<0)/np.sum(~np.isnan(F)) for F in np.array(FullOrder4,dtype=object)[labelsNAD2==1]]
print(stats.ttest_ind(A,A2))
PlotViolScat(A,np.clip(ColorCode['mPFC'],0,1),4)
PlotViolScat(A2,np.clip(ColorCode['mPFC']+0.2,0,1),4.2)


plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
plt.ylabel('Proportion of G1 in PL [\%]',fontsize=32)
plt.legend(fontsize=28,handletextpad=-0.5,markerscale=2,ncols = 2,loc=1,
           bbox_to_anchor=(0.8,1.2))


In [ ]:
j=0
i=0
R = RawDatNAD2[j][i]
TC = TimeCalcDatNAD2[j][i]
ST = ShockTimesNAD2[j][i]
F = FullOrder[i]
FA = FullOrderA[i]
fig,ax = plt.subplots(1,3,figsize=(12,4))
ax[0].plot(TC,(R)[F<0].T,alpha=0.08,c=ColorCode['mPFC'])
ax[0].plot(TC,(R)[F<0].T.mean(axis=-1),c=ColorCode['mPFC'],lw=3)
ax[0].set_xlim([ST[0]-10,ST[0]+10])
ax[0].set_xticks([ST[0]-10,ST[0],ST[0]+10],[-10,'Shock',10])
ax[0].set_ylim(-3, 15)
ax[1].plot(TC,(R)[FA>0].T,alpha=0.05,c=np.clip(ColorCode['mPFC']+0.2,0,1))
ax[1].plot(TC,(R)[FA>0].T.mean(axis=-1),c=np.clip(ColorCode['mPFC'],0,1),lw=3)
ax[1].set_xlim([ST[0]-10,ST[0]+10])
ax[1].set_xticks([ST[0]-10,ST[0],ST[0]+10],[-10,'Shock',10])
ax[1].get_yaxis().set_visible(False)
ax[1].spines['left'].set_visible(False)
ax[1].set_ylim(-3, 15)

ax[2].plot(TC,(R)[np.logical_and(F==0,FA==0)].T,alpha=0.05,c='gray')
ax[2].plot(TC,(R)[np.logical_and(F==0,FA==0)].T.mean(axis=-1),c='gray',lw=3)
ax[2].set_xlim([ST[0]-10,ST[0]+10])
ax[2].set_xticks([ST[0]-10,ST[0],ST[0]+10],[-10,'Shock',10])
ax[2].get_yaxis().set_visible(False)
ax[2].spines['left'].set_visible(False)
ax[2].set_ylim(-3, 15)
fig.suptitle('10 seconds before to 10 seconds after 1st shock \n z-score')
ax[0].set_xlabel('Time [s]',fontsize=24)
ax[1].set_xlabel('Time [s]',fontsize=24)
ax[2].set_xlabel('Time [s]',fontsize=24)

In [ ]:
i=0
CorrShock = []
CorrAShock = []
CorrOShock = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    FO = FullOrder[j]+FullOrderA[j]
    FO = FO[~np.isnan(Calc.sum(axis=1))]
    Calc = Calc[~np.isnan(Calc.sum(axis=1))]
    FO = FO[Calc.sum(axis=1)>0]
    Calc = Calc[Calc.sum(axis=1)>0]    
    TC = TimeCalcDatNAD2[j][i]
    TS = TimesNAD2[j][i]
    ST = ShockTimesNAD2[j][i]
    T = TimeCalcDatNAD2[j][i]
    D = DatNAD2[j][i][2]
    TD = DatNAD2[j][i][0]


    mask1 = np.logical_and(T>TS[0],T<TS[1])
    Calc = Calc[:,mask1]
    T = T[mask1]-TS[0]
    mask2 = np.logical_and(T>TD[0],T<TD[-1])

    Calc = Calc[:,mask2]
    FO = FO[Calc.sum(axis=1)>0]
    Calc = Calc[Calc.sum(axis=1)>0]
    T = T[mask2]

    f = sp.interpolate.interp1d(TD,D)
    D_int = f(T)
    Corrs = []
    for C in Calc:
        Corrs.append(np.corrcoef(D_int,C)[0,1])
    CorrShock.append(np.array(Corrs)[FO<0])
    CorrOShock.append(np.array(Corrs)[FO==0])
    CorrAShock.append(np.array(Corrs)[FO>0])
    
K = np.array([np.nanmean(C) for C in CorrShock])
K = K[~np.isnan(K)]
K2 = np.array([np.nanmean(C) for C in CorrAShock])
K2 = K2[~np.isnan(K2)]
K3 = np.array([np.nanmean(C) for C in CorrOShock])
K3 = K3[~np.isnan(K3)]
PlotViolScat(K,np.clip(ColorCode['mPFC'],0,1),1)
PlotViolScat(K2,np.clip(ColorCode['mPFC']+0.2,0,1),1.5)
PlotViolScat(K3,'gray',2)
plt.xticks([1,1.5,2],['G1','G2','Other'])
plt.axhline(0,c='k',ls='--')
plt.plot([2,2],[0.18,0.2],'k')
plt.plot(2,0.22,marker=(6, 2,0 ),ms=10,c='k')
plt.plot([1.5,1.5],[0.18,0.2],'k')

x = np.array([0.94,0.98, 1.02,1.06])+0.5
y = [0.22]*4
plt.plot(x, y, marker=(6, 2, 0), ms=10, c='k', linestyle='') 
plt.ylabel('Correlation with Freezing',fontsize=28)
plt.show()

In [ ]:
i=0
zPre = []
z1   = []
z2   = []
for j in range(len(labelsNAD2)):
    Calc = CalcDatNAD2[j][i]
    Order = FullOrder[j][~np.isnan(Calc.sum(axis=1))]
    Calc = Calc[~np.isnan(Calc.sum(axis=1))]
    Calc = Calc[Calc.sum(axis=1)>0]
    TC = TimeCalcDatNAD2[j][i]
    TS = TimesNAD2[j][i]
    ST = ShockTimesNAD2[j][i]
    D = DatNAD2[j][i]
    C1 = z_score(Calc[Order<0])
    C1_pre = C1[:,(TC>ST[0]-2)*(TC<ST[0])]
    C1_1 = C1[:,(TC>ST[0])*(TC<ST[0]+2)]
    C1_2 = C1[:,(TC>ST[1])*(TC<ST[1]+2)]
    zPre.append(np.mean(C1_pre))
    z1.append(np.mean(C1_1))
    z2.append(np.mean(C1_2))
zPre = np.array(zPre)
z1 = np.array(z1)
z2 = np.array(z2)
A = z1[labelsNAD2==2]
A2 = z1[labelsNAD2==1]

PlotViolScat(A,np.clip(ColorCode['mPFC'],0,1),1,lab='R+')
PlotViolScat(A2,np.clip(ColorCode['mPFC']+0.2,0,1),2,lab='R-')

A = z2[labelsNAD2==2]
A2 = z2[labelsNAD2==1]

PlotViolScat(A,np.clip(ColorCode['mPFC'],0,1),1.2)
PlotViolScat(A2,np.clip(ColorCode['mPFC']+0.2,0,1),2.2)

plt.ylabel('Avg. z-score 2s after shock',fontsize=28)
plt.xticks([1,1.2,2,2.2],['1st','2nd']*2,fontsize=20)

plt.legend(fontsize=28,handletextpad=-0.5,markerscale=2,ncols = 2,loc=1,
           bbox_to_anchor=(0.8,1.2))

In [ ]:
AntiO = []
SpikeO = []
AntiSpike = []
DB = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][0][~np.isnan(CalcDatNAD2[j][0].sum(axis=1))]
    O    = (FullOrder[j]+FullOrderA[j])[~np.isnan(CalcDatNAD2[j][0].sum(axis=1))].clip(-1,1)

    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T
    db_index = davies_bouldin_score(Xp.T, O)
    DB.append(db_index)

    #plt.title(LargNames[labelsNAD2[j]-1])
    X1 = Xp[:,O==1].mean(axis=-1)
    X2 = Xp[:,O==0].mean(axis=-1)
    X3 = Xp[:,O==-1].mean(axis=-1)
    maxdist = 0
    for x in Xp.T:
        for y in Xp.T:
            maxdist = max(np.linalg.norm(x-y),maxdist)
    AntiO.append(np.linalg.norm(X1-X2)/maxdist)
    AntiSpike.append(np.linalg.norm(X1-X3)/maxdist)
    SpikeO.append(np.linalg.norm(X2-X3)/maxdist)

    
AntiO = []
SpikeO = []
AntiSpike = []
DB1 = []
kk = 1
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][kk][~np.isnan(CalcDatNAD2[j][kk].sum(axis=1))]
    O    = FullOrder2[j][~np.isnan(CalcDatNAD2[j][kk].sum(axis=1))].clip(-1,1)

    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T

    colors = [ColorCode['D2Nac'],'gray',np.clip(ColorCode['D2Nac']+0.2,0,1)]
    cmap = ListedColormap(colors)
    
    db_index = davies_bouldin_score(Xp.T, O)
    DB1.append(db_index)
    X1 = Xp[:,O==1].mean(axis=-1)
    X2 = Xp[:,O==0].mean(axis=-1)
    X3 = Xp[:,O==-1].mean(axis=-1)
    maxdist = 0
    for x in Xp.T:
        for y in Xp.T:
            maxdist = max(np.linalg.norm(x-y),maxdist)
    AntiO.append(np.linalg.norm(X1-X2)/maxdist)
    AntiSpike.append(np.linalg.norm(X1-X3)/maxdist)
    SpikeO.append(np.linalg.norm(X2-X3)/maxdist)


AntiO = []
SpikeO = []
AntiSpike = []
DB2 = []
kk = 2
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][kk][~np.isnan(CalcDatNAD2[j][kk].sum(axis=1))]
    O    = FullOrder3[j][~np.isnan(CalcDatNAD2[j][kk].sum(axis=1))].clip(-1,1)

    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T

    colors = [ColorCode['D2Nac'],'gray',np.clip(ColorCode['D2Nac']+0.2,0,1)]
    cmap = ListedColormap(colors)
    
    db_index = davies_bouldin_score(Xp.T, O)
    DB2.append(db_index)
    X1 = Xp[:,O==1].mean(axis=-1)
    X2 = Xp[:,O==0].mean(axis=-1)
    X3 = Xp[:,O==-1].mean(axis=-1)
    maxdist = 0
    for x in Xp.T:
        for y in Xp.T:
            maxdist = max(np.linalg.norm(x-y),maxdist)
    AntiO.append(np.linalg.norm(X1-X2)/maxdist)
    AntiSpike.append(np.linalg.norm(X1-X3)/maxdist)
    SpikeO.append(np.linalg.norm(X2-X3)/maxdist)


AntiO = []
SpikeO = []
AntiSpike = []
DB3 = []
kk = 3
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][kk][~np.isnan(CalcDatNAD2[j][kk].sum(axis=1))]
    O    = FullOrder4[j][~np.isnan(CalcDatNAD2[j][kk].sum(axis=1))].clip(-1,1)

    X = Calc # data array of shape n_neuron by n_time_points
    X = z_score(X)
    pca = PCA(n_components=15)
    Xp = pca.fit_transform(X).T

    colors = [ColorCode['D2Nac'],'gray',np.clip(ColorCode['D2Nac']+0.2,0,1)]
    cmap = ListedColormap(colors)
    
    db_index = davies_bouldin_score(Xp.T, O)
    DB3.append(db_index)
    X1 = Xp[:,O==1].mean(axis=-1)
    X2 = Xp[:,O==0].mean(axis=-1)
    X3 = Xp[:,O==-1].mean(axis=-1)
    maxdist = 0
    for x in Xp.T:
        for y in Xp.T:
            maxdist = max(np.linalg.norm(x-y),maxdist)
    AntiO.append(np.linalg.norm(X1-X2)/maxdist)
    AntiSpike.append(np.linalg.norm(X1-X3)/maxdist)
    SpikeO.append(np.linalg.norm(X2-X3)/maxdist)
    
DR =np.vstack([np.array(DB)[labelsNAD2==2],np.array(DB1)[labelsNAD2==2],np.array(DB2)[labelsNAD2==2],np.array(DB3)[labelsNAD2==2]])
DS =np.vstack([np.array(DB)[labelsNAD2==1],np.array(DB1)[labelsNAD2==1],np.array(DB2)[labelsNAD2==1],np.array(DB3)[labelsNAD2==1]])
plt.errorbar([1,2,3,4],DR.mean(axis=1),stats.sem(DR,axis=1),
             c=ColorCode['mPFC']+0.2,label='R+',lw=3)
plt.errorbar([1,2,3,4],DS.mean(axis=1),stats.sem(DS,axis=1),
             c=np.clip(ColorCode['mPFC'],0,1),label='R-',lw=3)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
plt.ylabel('Davies-Bouldin Index \n for PC space',fontsize=28)
plt.legend(fontsize=28,loc=1)

PlotSig(2,2,8.5)
plt.plot([2,2],[8.3,8.1],lw=2,c='k')